In [15]:
# 读取excel文件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 读取excel文件
df = pd.read_excel("E:/研究生学习/研一上学期/高级应用统计/2023 第3次作业数据.xlsx", sheet_name='体脂率数据',header=0)
df.columns

Index(['brozek', 'siri', 'density', 'age', 'weight', 'height', 'adipos',
       'free', 'neck', 'chest', 'abdom', 'hip', 'thigh', 'knee', 'ankle',
       'biceps', 'forearm', 'wrist'],
      dtype='object')

In [16]:
# 删除siri，density，free三个属性
df = df.drop(['siri','density','free'],axis=1)

# 使用pandas对数据进行标准化
df = (df-df.mean())/df.std()


#使用标准线性回归，设置brozek为y，其他为x
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
lr = LinearRegression()
lr.fit(X_train,y_train)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)
print('MSE train:%.3f,test:%.3f'%(mean_squared_error(y_train,y_train_pred),mean_squared_error(y_test,y_test_pred)))



MSE train:0.258,test:0.227


In [5]:
import statsmodels.api as sm

# 使用statsmodels进行回归分析,剔除不显著的属性
X = df[:,1:]
y = df[:,0]





X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     50.52
Date:                Mon, 04 Dec 2023   Prob (F-statistic):           5.89e-63
Time:                        11:01:50   Log-Likelihood:                -183.39
No. Observations:                 252   AIC:                             396.8
Df Residuals:                     237   BIC:                             449.7
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.163e-17      0.033   1.28e-15      1.0

In [17]:
df.columns

Index(['brozek', 'age', 'weight', 'height', 'adipos', 'neck', 'chest', 'abdom',
       'hip', 'thigh', 'knee', 'ankle', 'biceps', 'forearm', 'wrist'],
      dtype='object')

In [18]:
def looper(limit):
    cols = [ 'age', 'weight', 'height', 'adipos', 'neck', 'chest', 'abdom',
       'hip', 'thigh', 'knee', 'ankle', 'biceps', 'forearm', 'wrist']
    for i in range(len(cols)):
        data1 = df[cols]
        x = sm.add_constant(data1) #生成自变量
        y = df['brozek'] #生成因变量
        model = sm.OLS(y, x) #生成模型
        result = model.fit() #模型拟合
        pvalues = result.pvalues #得到结果中所有P值
        pvalues.drop('const',inplace=True) #把const取得
        pmax = max(pvalues) #选出最大的P值
        if pmax>limit:
            ind = pvalues.idxmax() #找出最大P值的index
            cols.remove(ind) #把这个index从cols中删除
        else:
            return result
 
result = looper(0.10)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 brozek   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     101.6
Date:                Mon, 04 Dec 2023   Prob (F-statistic):           1.26e-68
Time:                        11:11:35   Log-Likelihood:                -185.16
No. Observations:                 252   AIC:                             386.3
Df Residuals:                     244   BIC:                             414.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.123e-17      0.032   9.67e-16      1.000      -0.064       0.064
age            0.1000      0.046      2.156      0.032       0.009       0.191
weight        -0.4260      0.119     -3.565      0.000      -0.661      -0.191
neck          -0.1167      0.064     -1.821      0.070      -0.243       0.010
abdom          1.1846      0.090     13.229      0.000       1.008       1.361
thigh          0.1421      0.073      1.952      0.052      -0.001       0.285
forearm        0.1351      0.045      3.028      0.003       0.047       0.223
wrist         -0.1687      0.057     -2.963      0.003      -0.281      -0.057
==============================================================================
Omnibus:                        4.566   Durbin-Watson:                   1.820
Prob(Omnibus):                  0.102   Jarque-Bera (JB):                2.888
Skew:                          -0.010   Prob(JB):                        0.236
Kurtosis:                       2.476   Cond. No.                         9.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
#使用LASSO回归,并输出各个属性的系数
from sklearn.linear_model import Lasso
alpha = [0.001,0.005,0.01,0.05,0.1]
all_coef = []
for i in alpha:
    lasso = Lasso(alpha=i)
    lasso.fit(X,y)

    # 将lasso的回归系数与标签对应起来
    lasso_coef = pd.DataFrame(lasso.coef_,index=X.columns)
    lasso_coef.loc['intercept'] = lasso.intercept_
    lasso_coef.loc['score']=lasso.score(X,y)
    lasso_coef.columns =[i]
    all_coef.append(lasso_coef)

# 将all_coef中的所有结果合并
result = pd.concat(all_coef,axis=1)
result


,0.001,0.005,0.010,0.050,0.100
age,9.135360e-02,8.806182e-02,8.505133e-02,6.022266e-02,1.593259e-02
weight,-2.945990e-01,-2.293387e-01,-1.387211e-01,-0.000000e+00,-0.000000e+00
height,-2.954226e-02,-4.069241e-02,-5.284320e-02,-6.818646e-02,-4.866341e-02
adipos,1.455529e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
neck,-1.357897e-01,-1.235692e-01,-1.082989e-01,-0.000000e+00,-0.000000e+00
chest,-1.989551e-02,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
abdom,1.206740e+00,1.141764e+00,1.066810e+00,8.175543e-01,7.139101e-01
hip,-1.737912e-01,-1.272982e-01,-7.637151e-02,-0.000000e+00,0.000000e+00
thigh,1.454415e-01,1.052256e-01,4.986306e-02,0.000000e+00,0.000000e+00
knee,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00


In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
X = df[:,1:]
y = df[:,0]
lasso.fit(X,y)
y_train_pred = lasso.predict(X_train)
y_test_pred = lasso.predict(X_test)
print('MSE train:%.3f,test:%.3f'%(mean_squared_error(y_train,y_train_pred),mean_squared_error(y_test,y_test_pred)))